## Notebook Plan

This notebook will create the scoring metrics used to measure innovation in our selected cities.

1. Create a new metric to measure the patent classification spread in a city
2. Create normalized (to 1) scores for each city. The metrics for these scores come from literature defined in the Progress Report. Contain patents, citations and classifications.

In [1]:
import requests
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

import matplotlib as mplib
import matplotlib.pyplot as plt
from datetime import datetime
import copy
import ast
from collections import Counter
import itertools
from ast import literal_eval
import pandas_profiling
import time

from sklearn.cluster import KMeans
from sklearn import preprocessing

import seaborn as sns

%matplotlib inline

### Read in Patent data and drop non-US based cities

In [2]:
df = pd.read_csv('Data/Cleaned Data/patents_data_cleaned_all_years_new.csv')

In [3]:
df = df.loc[~df.State.isna()]

In [4]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
print(df.shape)
df.head()

(38593, 40)


,City,Inv_to_Assignee_ratio,Patents,State,Year,assignee_IPC_A,assignee_IPC_B,assignee_IPC_C,assignee_IPC_D,assignee_IPC_E,...,inventor_IPC_G,inventor_IPC_H,inventor_patents,inventor_pats_cited,inventor_pats_cited_ratio,pt_design,pt_plant,pt_reissue,pt_statutory invention registration,pt_utility
0,New York,0.055923,4077,NY,1976,0.121036,0.157771,0.280063,0.042229,0.023391,...,0.111111,0.079365,228,64,0.280702,0.035075,0.0,0.008585,0.0,0.955359
1,Washington,0.021840,1511,DC,1976,0.055236,0.136544,0.160407,0.012815,0.013699,...,0.142857,0.160714,33,9,0.272727,0.000662,0.0,0.000000,0.0,0.998676
2,Pittsburgh,0.296154,1300,PA,1976,0.024614,0.216699,0.286680,0.015444,0.028958,...,0.139219,0.156197,385,141,0.366234,0.005385,0.0,0.005385,0.0,0.987692
3,Chicago,0.305112,1252,IL,1976,0.136054,0.242282,0.179487,0.048142,0.030874,...,0.122625,0.136442,382,122,0.319372,0.062300,0.0,0.015176,0.0,0.922524
4,Stamford,0.118511,1021,CT,1976,0.077848,0.200633,0.191139,0.041772,0.005063,...,0.232044,0.099448,121,46,0.380165,0.045054,0.0,0.006856,0.0,0.948090


In [5]:
df.columns

Index(['City', 'Inv_to_Assignee_ratio', 'Patents', 'State', 'Year',
       'assignee_IPC_A', 'assignee_IPC_B', 'assignee_IPC_C', 'assignee_IPC_D',
       'assignee_IPC_E', 'assignee_IPC_F', 'assignee_IPC_G', 'assignee_IPC_H',
       'assignee_pats_cited', 'assignee_pats_cited_ratio', 'assignee_type_2',
       'assignee_type_3', 'assignee_type_4', 'assignee_type_5',
       'assignee_type_6', 'assignee_type_7', 'assignee_type_8',
       'assignee_type_9', 'city_state', 'inventor_IPC_A', 'inventor_IPC_B',
       'inventor_IPC_C', 'inventor_IPC_D', 'inventor_IPC_E', 'inventor_IPC_F',
       'inventor_IPC_G', 'inventor_IPC_H', 'inventor_patents',
       'inventor_pats_cited', 'inventor_pats_cited_ratio', 'pt_design',
       'pt_plant', 'pt_reissue', 'pt_statutory invention registration',
       'pt_utility'],
      dtype='object')

### Create Classifcation Spread for both assigned and invented patents

In [6]:
#Spread of patents across classifications
# Higher return value is bad for innovation. 
def IPC_spread(row):
    assignee_vals = [row['assignee_IPC_A'], row['assignee_IPC_B'], row['assignee_IPC_C'], row['assignee_IPC_D'],
                     row['assignee_IPC_E'], row['assignee_IPC_F'], row['assignee_IPC_G'], row['assignee_IPC_H']]
    max_val = max(assignee_vals)
    min_val = min(assignee_vals)
    return(max_val - min_val)

#Spread of patents across classifications
# Higher return value is bad for innovation. 
def IPC_inv_spread(row):
    inventor_vals = [row['inventor_IPC_A'], row['inventor_IPC_B'], row['inventor_IPC_C'], row['inventor_IPC_D'],
                     row['inventor_IPC_E'], row['inventor_IPC_F'], row['inventor_IPC_G'], row['inventor_IPC_H']]
    max_inv_val = max(inventor_vals)
    min_inv_val = min(inventor_vals)
    return(max_inv_val - min_inv_val)

In [7]:
df['IPC_assig_spread'] = df.apply(lambda x: IPC_spread(x), axis = 1)
df['IPC_inv_spread'] = df.apply(lambda x: IPC_inv_spread(x), axis = 1)

### Create 7 scores that will be used to analyze innovation

In [8]:
min_max_scaler = preprocessing.MinMaxScaler()
scored_df = []

In [9]:
for year in range(1976,2015):
    temp_df = df.loc[df.Year == year]
    
    # Inventor Citations
    temp_df['Score1'] = temp_df['inventor_pats_cited']
    temp_df['Score1'] = temp_df.Score1**(1/np.log(temp_df.Score1.max()))

    # Inventor and Assignee Citations
    temp_df['Score2'] = temp_df['inventor_pats_cited'] + temp_df['assignee_pats_cited']
    temp_df['Score2'] = temp_df.Score2**(1/np.log(temp_df.Score2.max()))

    # Score 1 with classification spread
    #temp_df['Score3'] = temp_df['Score1'] / (temp_df['IPC_inv_spread'])

    # Score 2 with classification spread
    #temp_df['Score4'] = temp_df['Score2'] / (temp_df['IPC_inv_spread'] + temp_df['IPC_assig_spread'])

    # Number of invented patents
    temp_df['Score5'] = temp_df['inventor_patents']
    temp_df['Score5'] = temp_df.Score5**(1/np.log(temp_df.Score5.max()))
    
    # Number of assigned patents
    temp_df['Score6'] = temp_df['Patents']
    temp_df['Score6'] = temp_df.Score6**(1/np.log(temp_df.Score6.max()))


    #temp_df = temp_df[~temp_df.isin([np.nan, np.inf, -np.inf]).any(1)]
    #temp_df.dropna(inplace = True)
    #temp_df['Score1'] = min_max_scaler.fit_transform(temp_df[['Score1']])
    #temp_df['Score2'] = min_max_scaler.fit_transform(temp_df[['Score2']])
    #temp_df['Score3'] = min_max_scaler.fit_transform(temp_df[['Score3']])
    #temp_df['Score4'] = min_max_scaler.fit_transform(temp_df[['Score4']])
    #temp_df['Score5'] = min_max_scaler.fit_transform(temp_df[['Score5']])

    # Total patents 
    temp_df['Score7'] = temp_df['Score5'] + temp_df['Score6']
    
    
    temp_df = temp_df[['city_state', 'Year', 'Score1', 'Score2', 'Score5', 'Score6', 'Score7']]
    scored_df.append(temp_df)
    del temp_df

/Users/rohun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/rohun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/rohun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

In [10]:
final_df = pd.concat(scored_df, ignore_index = True)

In [11]:
final_df.head()

,city_state,Year,Score1,Score2,Score5,Score6,Score7
0,"('New York', 'NY')",1976,2.317268,2.718282,2.360150,2.718282,5.078432
1,"('Washington', 'DC')",1976,1.558923,2.219101,1.738496,2.412346,4.150842
2,"('Pittsburgh', 'PA')",1976,2.718282,2.425776,2.564047,2.369092,4.933139
3,"('Chicago', 'IL')",1976,2.639930,2.393199,2.560876,2.358395,4.919271
4,"('Stamford', 'CT')",1976,2.167678,2.186224,2.135113,2.301236,4.436350


In [12]:
final_df.describe()

,Year,Score1,Score2,Score5,Score6,Score7
count,38593.000000,38593.000000,38593.000000,38593.000000,38593.000000,38593.000000
mean,1994.915555,1.516470,1.627062,1.531346,1.476043,3.007389
std,11.242758,0.552607,0.378332,0.395867,0.248663,0.560697
min,1976.000000,0.000000,0.000000,0.000000,1.141284,1.141284
25%,1985.000000,1.349023,1.430782,1.368124,1.311045,2.716092
50%,1995.000000,1.617979,1.639855,1.560107,1.410793,2.974498
75%,2005.000000,1.849058,1.854163,1.750397,1.584629,3.291353
max,2014.000000,2.718282,2.718282,2.718282,2.718282,5.404603


In [13]:
final_df.head()

,city_state,Year,Score1,Score2,Score5,Score6,Score7
0,"('New York', 'NY')",1976,2.317268,2.718282,2.360150,2.718282,5.078432
1,"('Washington', 'DC')",1976,1.558923,2.219101,1.738496,2.412346,4.150842
2,"('Pittsburgh', 'PA')",1976,2.718282,2.425776,2.564047,2.369092,4.933139
3,"('Chicago', 'IL')",1976,2.639930,2.393199,2.560876,2.358395,4.919271
4,"('Stamford', 'CT')",1976,2.167678,2.186224,2.135113,2.301236,4.436350


In [14]:
to_drop = []
for i in range(len(final_df)):
    city_state = final_df.iloc[i]['city_state']
    try:
        x = city_state.split("'")[3]
    except:
        to_drop.append(i)
        print(str(i) + ": " + str(city_state))

23731: (None, 'DE')
24473: (None, 'DE')
24674: (None, 'MA')
24774: (None, 'NJ')
24776: (None, 'TX')
25207: (None, 'DE')
25604: (None, 'NJ')
26342: (None, 'DE')
26800: (None, 'NJ')
27473: (None, 'DE')
28427: (None, 'DE')
28665: (None, 'NJ')
29281: (None, 'DE')
30196: (None, 'DE')
31421: (None, 'DE')
32161: (None, 'DE')
33352: (None, 'DE')
33661: (None, 'NJ')
34647: (None, 'DE')
36568: (None, 'KY')


In [15]:
final_df.drop(to_drop, inplace = True)

In [16]:
final_df['City'] = final_df.city_state.apply(lambda x: x.split("'")[1])
final_df['State'] = final_df.city_state.apply(lambda x: x.split("'")[3])
final_df['city_state'] = final_df['City'] + "_" + final_df['State']
final_df['city_state'] = final_df.city_state.apply(lambda x: str.lower(x.replace(' ', '')))
final_df.head()

,city_state,Year,Score1,Score2,Score5,Score6,Score7,City,State
0,newyork_ny,1976,2.317268,2.718282,2.360150,2.718282,5.078432,New York,NY
1,washington_dc,1976,1.558923,2.219101,1.738496,2.412346,4.150842,Washington,DC
2,pittsburgh_pa,1976,2.718282,2.425776,2.564047,2.369092,4.933139,Pittsburgh,PA
3,chicago_il,1976,2.639930,2.393199,2.560876,2.358395,4.919271,Chicago,IL
4,stamford_ct,1976,2.167678,2.186224,2.135113,2.301236,4.436350,Stamford,CT


In [49]:
df_1997_2014 = final_df.loc[(final_df.Year > 1996) & (final_df.Year < 2015)]

### Write to temp file to file

In [50]:
df_1997_2014.to_csv('patent_scores_1997_2014.csv')

### Write to file

In [51]:
final_df.to_csv('cities_with_scores.csv')